# SM from IRRmodel into WCM, both with PSO on KGE

The v5 code is a final, cleaned and commented version.

In [6]:
from modules.funcs import *
from modules.funcs_pso import *
from IRRI_WCM.IRRI_WCM_model import *

In [7]:
def pso_calib_irri(PAR):
    """Ausiliary function for PSO optimization"""
    global inputs
    global irri
    n_particles = PAR.shape[0]
    err = np.zeros(n_particles)
    for i in range(n_particles):
        WW,IRR,sig0,KGE = IRR_WCM(PAR[i], inputs, irri)
        err[i] = 1 - KGE
    return err

# Global

In [8]:
filename = f'irr_obs_'

# Input data

Input data formatting convention:
- ausiliary variables for extraction of data (directory name, file name, etc...)
- extraction into pd dataframe
- cleaning, resampling: drop unnecessary columns, set index to daily DateIndex
- extract variables [RIGHT HERE! :) ]

In [32]:
print('Starting...\n'+
      '#-------------------------------------------------------------\n'+
      'Use of satellite-derived SM is provided for comparison, not calibration.\n')
verbose = True if input("Verbose data extraction? (Describe datasets/files) [y/n]")=='y' else False

Starting...
#-------------------------------------------------------------
Use of satellite-derived SM is provided for comparison, not calibration.



Verbose data extraction? (Describe datasets/files) [y/n] y


In [33]:
#-----------------------------------------------------------------------------
# Data reading
#-----------------------------------------------------------------------------

# Field data from TEST_SITE

# Extracted data:
# - EPOT = potential evapotranspiration

namesite = 'ITALY_BUDRIO'
siteID = '5'
namefig = namesite+'_'+siteID

sitedf = xr.open_dataset(f'IRRmodel\TEST_SITE\TEST_SITE_{namesite}.nc',
                         engine='netcdf4').to_dataframe();
sitedf = sitedf.set_index('Time_days')
if verbose: sitedf.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1037 entries, 2015-01-01 to 2017-11-02
Data columns (total 60 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Irrigation_1        627 non-null    float64
 1   Rainfall_1          1037 non-null   float64
 2   ET_1                1037 non-null   float64
 3   PET_1               1037 non-null   float64
 4   SSM_ASCAT_1         1009 non-null   float64
 5   SSM_CCI_combined_1  998 non-null    float64
 6   SSM_CCI_active_1    972 non-null    float64
 7   SSM_CCI_passive_1   925 non-null    float64
 8   SSM_SMAP_1          609 non-null    float64
 9   SSM_SMOS_1          598 non-null    float64
 10  SSM_THEIA_1         176 non-null    float64
 11  SSM_RT1_1           241 non-null    float64
 12  Irrigation_2        627 non-null    float64
 13  Rainfall_2          1037 non-null   float64
 14  ET_2                1037 non-null   float64
 15  PET_2               1037 non-null   f

In [34]:
# Satellite and veg data (S-1) from cleaned data table
df = pd.read_csv('Data\data_in.csv', delimiter='\t', index_col=0);
df.Date = df.Date.apply(lambda x : pd.to_datetime(x))
df = df.set_index('Date')
if verbose: df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 107 entries, 2017-04-04 to 2017-11-02
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SM_irri[-]       107 non-null    float64
 1   LAI[m2/m2]       107 non-null    float64
 2   RT1[-]           107 non-null    float64
 3   Angle[°]         107 non-null    float64
 4   DateTime_sat     107 non-null    object 
 5   Geometry         107 non-null    object 
 6   Orb              107 non-null    int64  
 7   Pass             107 non-null    object 
 8   VH[lin]          107 non-null    float64
 9   VH_var[lin]      107 non-null    float64
 10  VV[lin]          107 non-null    float64
 11  VV_var[lin]      107 non-null    float64
 12  VV[dB]           107 non-null    float64
 13  VH[dB]           107 non-null    float64
 14  VV_var[dB]       107 non-null    float64
 15  VH_var[dB]       107 non-null    float64
 16  SWC[m3/m3]       103 non-null    float64
 1

In [ ]:
# Budrio field data from Platinum tables
platinum = pd.ExcelFile('Data\Platinum_Budrio.xlsx', engine='openpyxl')
platinum = platinum.parse('2017_1h')
platinum['Ora_1'] = pd.to_datetime(platinum['Ora'].astype('str')).apply(lambda x: x.time())
platinum['Data_1'] = pd.to_datetime(platinum['Data'].astype('str')).apply(lambda x: x.date())
platinum['Date'] = platinum.apply(lambda r : dtt.datetime.combine(r['Data_1'],r['Ora_1']),1)
platinum = platinum.drop(['ID', 'Data_1', 'Ora_1', '214Pb[cps]'],axis=1)
#platinum_cut = pd.concat([platinum.loc[(platinum.Data<'2017-05-14')],platinum.loc[(platinum.Data>'2017-06-16')]])

In [ ]:
# Dates
D_0 = sitedf.Time_days; set1 = {x for x in D_0}
D_1 = platinum['Data']; set2 = {x for x in D_1}
d = np.sort(np.array([*set1.intersection(set2)])) # dates, full

d_sat = [pd.Timestamp(df.index[i]) for i in range(len(df.index))]
set1 = {x for x in d}; set2 = {x for x in d_sat}
d_sat = np.sort(np.array([*set1.intersection(set2)])) # dates, passage of sat

In [ ]:
# Backscattering
# t_deg = df['Angle[°]'].values
t_deg = [np.mean(df.loc[df.Orb==95]['Angle[°]'].values) for i in range(len(df.Orb.values))]
obs = df.loc[d_sat]['VV_norm[dB]'].values
obs_VH = df.loc[d_sat]['VH_norm[dB]'].values
rt1 = df.loc[d_sat]['RT1[-]']


# Vegetation indexes
LAI = df.loc[d_sat]['LAI[m2/m2]'].values
cr = (db_lin(obs_VH)/db_lin(obs))

df_NDVI = pd.read_csv(f'Data\\NDVI_GEE.csv', delimiter = "\t")
df_NDVI.Datetime = df_NDVI.Datetime.apply(lambda x:pd.Timestamp(x))
NDVI = df_NDVI.set_index('Datetime').loc[d_sat]['NDVI'].values

P = platinum.resample('1D',on='Date').sum().loc[d]['Pioggia[mm]'].values
EPOT = sitedf.loc[d][f'PET_{siteID}'].values # potential evapotranspiration (measured)
IRRobs = platinum.resample('1D',on='Date').sum().loc[d]['Irrigazione[mm]'].values
Wobs_gap = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].values
Wobs = platinum.resample('1D',on='Date').mean().loc[d]['SWC[m3/m3]'].interpolate(method='linear').values
WWobs_gap = (Wobs_gap-np.min(Wobs))/(np.max(Wobs)-np.min(Wobs))
WWobs = norm(Wobs)

index = input('Please provide name of vegetation index to use as input. [Options: LAI, cr, vh, NDVI]')
if index=='LAI': veg=LAI; label_veg='LAI[-]'
elif index=='NDVI': veg=NDVI; label_veg='NDVI[-]'
elif index=='cr': veg=cr; label_veg=r'$\sigma^0$ cross ratio VH/VV [-]'
elif index=='vh': veg=db_lin(obs_VH); label_veg=r'$\sigma^0$ [VH] [dB]'
else: raise NameError('Please select one of the available options')

Starting...
#-------------------------------------------------------------
Use of satellite-derived SM is provided for comparison, not calibration.



Please provide name of vegetation index to use as input. [Options: LAI, cr, vh, NDVI] NDVI
